In [1]:
import wget
import ssl
import os
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# Enter NCRB Individual PDF url
pdf_url = 'https://ncrb.gov.in/sites/default/files/adsi_reports_previous_year/Table-1A.1_2019.pdf'
fileName = pdf_url[67:]
try:
    #Delete the file to redownload it
    os.remove(fileName)
except FileNotFoundError as e:
    #Ignore in case FileNotFound Error occurs
    pass
# Download report
fileName = wget.download(pdf_url,fileName)

In [3]:
import numpy as np
import pandas as pd
import camelot

In [4]:
fileName

'Table-1A.1_2019.pdf'

In [5]:
tables = camelot.read_pdf(fileName,
                          flavor='stream',
                          table_areas=[
                              "24.52943457189015,722.7821979832302,575.7202584814218,67.68997164783696"
                              ],
                          columns=[
                              "51.94468497576737,150.06242326332796,191.4257835218094,224.1316962843296,252.98985460420036,286.65770597738293,315.36361873990313,353.0314701130857,383,408,445.3775767366721,483.8551211631665,518,547.3430694668822"
                              ],
                          split_text=True,
                          pages="all"
                         )

In [6]:
tables

<TableList n=2>

In [7]:
def printnAllTables(tables):
    for i in range(len(tables)):
        print("Page No: ",(i+1))
        print(tables[i].df)
        print()

In [8]:
# Incase we're intersted in seeing the original scraped tables.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

In [9]:
# Removes Garbage Rows that may have been detected Before the 'STATES' row
# AND
# Removes Garbage Rows that may have been detected after the 'TOTAL (ALL INDIA)' row
def clean(table):
    while(table.df.iloc[0,1] != 'STATES' and table.df.iloc[0,1] != 'AGRA'):
        table.df = table.df.iloc[1:]
    while(table.df.iloc[-1,1] != 'TOTAL (ALL INDIA)' and table.df.iloc[-1,1] != 'TOTAL (CITIES)'):
        table.df = table.df.iloc[:-1]
    # The Return line is unnecessary due to all changes being reflected in the original references
    return table

In [10]:
# Splits the second column into 1st column index and 2nd column test when faulty/required
def split_by_first_space(df, strIndex, stpIndex):
    df = df.iloc[strIndex:stpIndex+1]
    for index, each in df.iterrows():
        space = each[1].find(' ')
        each[0] = each[1][:space]
        each[1] = each[1][space:]

In [11]:
def mergeRows(df, strIndex, stpIndex):
    DF = df.loc[strIndex:stpIndex]
    df = DF.T
    l = []
    for index, each in df.iterrows():
        output = ''
        for ind, cell in each.iteritems():
            output += (str(cell) + ' ')
#         if(output == ''):
#             continue
        l.append(output.strip())
    l = pd.Series(l,name=str(strIndex))
    DF.loc[strIndex] = l

In [12]:
for each_table in tables:
    clean(each_table)

In [13]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

In [14]:
def set_Column(df, state):
    column_list = [
        ("Sl. No.",''),
        ("State/UT" if(state) else "City",''), 
        ('Road Accidents','2018'),
        ('Road Accidents','2019'),
        ('Road Accidents','% Variation'),
        ('Railway Accidents','2018'),
        ('Railway Accidents','2019'),
        ('Railway Accidents','% Variation'),
        ('Railway Crossing Accidents','2018'),
        ('Railway Crossing Accidents','2019'),
        ('Railway Crossing Accidents','% Variation'),
        ('Total Traffic Accidents','2018'),
        ('Total Traffic Accidents','2019'),
        ('Total Traffic Accidents','% Variation'),
        ('% Share','')
    ]
    df.columns = pd.MultiIndex.from_tuples(column_list)

In [15]:
for i in range(len(tables)):
    state = not i
    set_Column(tables[i].df, state)

In [16]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

# OR Print a simgle one according to your choice
# tables[23].df

In [17]:
# Reset Index before saving file, for better formatting in RAW CSV
tables[0].df.to_csv("Cases Reported and Percentage Change in Traffic Accidents during 2019 over 2018(StateUT-wise).csv",index=False)
tables[1].df.to_csv("Cases Reported and Percentage Change in Traffic Accidents during 2019 over 2018(City-wise).csv",index=False)